<a href="https://colab.research.google.com/github/galnov/hello-world/blob/dataset/Binary_Image_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we'll create a binary image classifier, train it and use it to classify a given image

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Check if GPU is available
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) == 0:
    print("No GPU devices available. Training on CPU.")
else:
    print("GPU is available. Training on GPU.")

In [ ]:
!pip install keras-facenet


Creating the model

In [ ]:
import numpy as np
from keras_facenet import FaceNet
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Load the FaceNet model
facenet = FaceNet()

# Freeze the layers in the FaceNet model
for layer in facenet.model.layers:
    layer.trainable = False

# Adding a binary classifier on top of the FaceNet model
x = Flatten()(facenet.model.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

# Create a new model
model = Model(inputs=facenet.model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy',
              metrics=['accuracy'])
#model.summary()

Uploading training data - run the following cell just once!

In [ ]:
!git clone https://github.com/galnov/hello-world
%cd hello-world
!git checkout dataset

Training

In [ ]:
# Set up data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'path/to/training_data',
    target_size=(160, 160),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'path/to/training_data',
    target_size=(160, 160),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Plot training history
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Save the trained model
model.save('image_classifier_model.h5')

Loading the model and predicting

In [ ]:
# Load the model for future use
loaded_model = models.load_model('image_classifier_model.h5')

# Now, you can use loaded_model.predict() to make predictions on new data.
